## Minmax Algorithm for  Tic-Tac-Toe

In [2]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt

playerA = 1
playerB = -1
empty = 0
optimal_move = '*'

def evaluate(state):
    """
    Evaluate the board state to return a score.
    +10 for a win for MAXIMIZER, -10 for a win for MINIMIZER, 0 for a draw or unfinished game.
    """
    for row in state:
        if all(cell == playerA for cell in row):
            return 10
        if all(cell == playerB for cell in row):
            return -10

    for col in range(len(state)):
        if all(row[col] == playerA for row in state):
            return 10
        if all(row[col] == playerB for row in state):
            return -10

    if all(state[i][i] == playerA for i in range(len(state))):
        return 10
    if all(state[i][i] == playerB for i in range(len(state))):
        return -10

    if all(state[i][len(state) - 1 - i] == playerA for i in range(len(state))):
        return 10
    if all(state[i][len(state) - 1 - i] == playerB for i in range(len(state))):
        return -10
    return 0

def game_over(state):
    """
    Check if the game is over due to a win or draw.
    """
    return evaluate(state) != 0 or all(cell != empty for row in state for cell in row)

def valid_moves(state):
    """
    Return a list of valid moves (empty cells) as (row, col) tuples.
    """
    return [(r, c) for r in range(len(state)) for c in range(len(state[r])) if state[r][c] == empty]

def minimax(state, depth, player):
    """
    Minimax algorithm implementation that returns all optimal moves for the current state.
    """
    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [[], score]  # Return an empty list of moves and the score

    if player == playerA:
        best = [[], -math.inf]  # Initialize with worst score for maximizer
    else:
        best = [[], math.inf]  # Initialize with worst score for minimizer

    for move in valid_moves(state):
        row, col = move
        # Make the move
        state[row][col] = player
        # Recursively call minimax for the opponent
        _, score = minimax(state, depth - 1, -player)
        state[row][col] = empty

        if player == playerA:
            if score > best[1]:  # Found a new better score
                best = [[move], score]  # Update with the new best move
            elif score == best[1]:  # Found another move with the same best score
                best[0].append(move)
        else:
            if score < best[1]:  # Found a new better score
                best = [[move], score]  # Update with the new best move
            elif score == best[1]:  # Found another move with the same best score
                best[0].append(move)

    return best


def print_board(state):
    """
    Print the Tic Tac Toe board.
    """
    symbols = {playerA: 'O', playerB: 'X', empty: ' ',optimal_move :'*'}
    for row in state:
        print(" | ".join(symbols[cell] for cell in row))
        print("-" * 5)



board = [
    [1, 0, 1],
    [-1, -1, 0],
    [0, 0, 0]
]

print("Initial Board:")
print_board(board)
player = playerA

best_moves, _ = minimax(board, depth=8, player=player)

row = [move[0] for move in best_moves]
col = [move[1] for move in best_moves]
for i in range(len(row)):

    board[row[i]][col[i]] = optimal_move

print("Best move for X:")
print_board(board)



Initial Board:
O |   | O
-----
X | X |  
-----
  |   |  
-----
Best move for X:
O | * | O
-----
X | X | *
-----
  |   |  
-----


In [3]:
optimal_moves_cache = {}

def collect_optimal_moves(initial_state, depth,player):
    """
    Calls the minimax function for the given initial state and stores the results in a dictionary.
    """
    def traverse_and_store(state, current_depth, current_player):
        state_tuple = list(tuple(row) for row in state)
        
        # Check if the result is already cached
        if state_tuple not in optimal_moves_cache:
            result,_ = minimax(state, current_depth, current_player)
            optimal_moves_cache[state_tuple] = result

            # Recursively traverse all valid moves and store their results
            for move in valid_moves(state):
                row, col = move
                state[row][col] = current_player  # Make the move
                traverse_and_store(state, current_depth - 1, -current_player)
                state[row][col] = empty  

    traverse_and_store(initial_state, depth,player)
initial_state = [
    [empty, empty, empty],
    [empty, empty, empty],
    [empty, empty, empty]
]

# Call the function with the initial state, desired depth, and starting player
collect_optimal_moves(initial_state, depth=9,player=playerA)


## Test the percentage of draws in the game 

We let the two players play additional rounds of the game to analyze the percentage of draws.

In [16]:
def play_game_with_optimal_moves(N):
    res = [0, 0, 0]
    # Make a copy of the initial state
    for _ in range(N):
        current_player = 1
        current_state = [row[:] for row in initial_state]
        while not game_over(current_state):
            state_tuple = tuple(tuple(row) for row in current_state)

            # Check if the current state exists in the cache

            optimal_moves = optimal_moves_cache[state_tuple]

            # Play the first optimal move
            move = random.choice(optimal_moves)
            row, col = move
            current_state[row][col] = current_player

            # Switch to the other player
            current_player = -current_player

            # Check for a draw condition (if the board is full and no winner)
        if evaluate(current_state) == 10:
            res[1] += 1
        elif evaluate(current_state) == -10:
            res[2] += 1
        else:
            res[0] += 1

    return res


N = 100
print(f'Percentage of draw: {play_game_with_optimal_moves(N)[0]/N*100}%.')

Percentage of draw: 100.0%.
